In [ ]:
import pandas as pd
import numpy as np

file_path = 'insert data here'  # Replace with your file path
data = pd.read_excel(file_path, sheet_name='Sheet1')
print(data.head(5))

In [ ]:
columns_to_drop = [
    'Submission Date', 'Name', 'Age', 'Gender',
    'Phone Number', 'Email', 'Date Signed','Allergies','Any Additional Preferences or Requirements','Preferred Location/Neighborhood'
]
drop_unneeded_cols_data = data.drop(columns=columns_to_drop, axis=1)

print(drop_unneeded_cols_data.head())


In [ ]:
import pandas as pd

def one_hot_encode_column(dataframe, column):
    unique_values = dataframe[column].dropna().str.split('\n').explode().unique()
    
    for value in unique_values:
        dataframe[value] = dataframe[column].apply(lambda x: 1 if pd.notna(x) and value in x.split('\n') else 0)
    
    dataframe.drop(column, axis=1, inplace=True)

columns_to_encode = [ 'Hobbies', 'Sports']
for column in columns_to_encode:
    one_hot_encode_column(drop_unneeded_cols_data, column)


In [ ]:
def one_hot_encode_single_selection_columns(dataframe):
    
    single_selection_columns = dataframe.select_dtypes(include=['object']).columns
    
    dataframe = pd.get_dummies(dataframe, columns=single_selection_columns, prefix=single_selection_columns)
    
    return dataframe

one_hot_encoded_data = one_hot_encode_single_selection_columns(drop_unneeded_cols_data)




In [ ]:
def calculate_compatibility_with_weights(person1, person2, special_columns, important_columns_weights):
    p1 = np.array(person1,dtype=int)
    p2 = np.array(person2,dtype=int)
    score = 0
    for i in range(len(p1)):
        if i in special_columns:
            if p1[i] == 1 or p2[i] == 1:
                score += 1
        elif i in important_columns_weights:
            if p1[i] == 1 and p2[i] == 1:
                score += important_columns_weights[i]
        else:
            score += p1[i] * p2[i]
    return score

special_columns = [34,38, 40, 45, 50, 53, 55, 58, 66]  # Example for "Preferred Lifestyle (Smoking)" = 54
important_columns_weights = {
    32: 1.5,  # Preferred Gender of Roommate_Female
    33: 1.5, # Preferred Gender of Roommate_Male
    34: 1.5, # Preferred Gender of Roommate_No Preference, getting skipped currently since its a no preference col
       
}


In [ ]:
def list_scores_for_person_weighted(dataframe, person_index, original_data):
    scores = []
    for i in range(len(dataframe)):
        if i != person_index:  
            score = calculate_compatibility_with_weights(dataframe.iloc[person_index], dataframe.iloc[i],special_columns,important_columns_weights)
            scores.append((original_data.iloc[i]['Name'], score))
    scores.sort(key=lambda x: x[1], reverse=True)
    return scores

person_index = 3  
all_scores2 = list_scores_for_person_weighted(one_hot_encoded_data, person_index, data)

person_name = data.iloc[person_index]['Name'] 
print(f"Compatibility Scores for {person_name} (sorted):")
for other_person_name, score in all_scores2:
    print(f"{other_person_name}: {score}")


In [ ]:
one_hot_encoded_data.to_csv('one_hot_encoded_data.csv', index=False)
print("Data saved to 'one_hot_encoded_data.csv'")